# 1. Idea
our problem can be seen as how to obtain the difference of two clusters.
I intend to use Silhouette index to calculate the difference

In [2]:
import pandas as pd

trajectory = pd.read_csv('/mnt/c/Users/ouyangkang/Desktop/trajectory-0719.csv', index_col=0)
trajectory.columns = [f"Celltype_{i}" for i in range(trajectory.shape[1])]
trajectory.tail()

,Celltype_0,Celltype_1,Celltype_2,Celltype_3,Celltype_4,Celltype_5,Celltype_6,Celltype_7
lineage,,,,,,,,
SM_lineage,myoblasts,muscle primordium,muscle system primordium,somatic muscle primordium,somatic muscle,NaN,NaN,NaN
VM_lineage,myoblasts,muscle primordium,muscle system primordium,undifferentiated muscle,visceral muscle primordium,visceral muscle,NaN,NaN
FB_lineage,fat body primordium,early-developing fat body,developing fat body,late-developing fat body,mature fat body,NaN,NaN,NaN
LG_lineage,plasmatocyte primordium,plasmatocytes,lymph gland,NaN,NaN,NaN,NaN,NaN
CB_lineage,cardiogenic mesoderm,dorsal vessel primordium,cardioblast,NaN,NaN,NaN,NaN,NaN


In [3]:
cell_connection = []

for lineage in trajectory.iterrows():
    for i in range(len(lineage[1].dropna()) - 1):
        cell_connection.append([lineage[1][i], lineage[1][i+1]])
    
pd.DataFrame(cell_connection, columns=['Celltype_1', 'Celltype_2']).to_csv('/home/ouyangkang/data/trajectory_graph/celltype_connection.csv', index=None)

In [4]:
import scanpy as sc

sc_data = sc.read_h5ad('/mnt/c/Users/ouyangkang/Desktop/convert_sc_data.h5ad')
sc_data

AnnData object with n_obs × n_vars = 238242 × 16725
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'cell', 'pANN', 'library', 'RNA_snn_res.0.5', 'seurat_clusters', 'collected_time', 'infer_age', 'infer_age_int', 'tissue_old', 'sub_cluster_number', 'celltype', 'tissue_update', 'germ_layer', 'subtype'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    obsm: 'X_harmony', 'X_pca', 'X_scvi', 'X_umap'

## 2. Using tow cell type to try to calculate difference between its

In [3]:
import numpy as np

sample_data = sc_data[np.in1d(sc_data.obs['celltype'], ['Malpighian tubule anlage', 'Malpighian tubule primordium'])]

In [4]:
sample_data

View of AnnData object with n_obs × n_vars = 2025 × 16725
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'cell', 'pANN', 'library', 'RNA_snn_res.0.5', 'seurat_clusters', 'collected_time', 'infer_age', 'infer_age_int', 'tissue_old', 'sub_cluster_number', 'celltype', 'tissue_update', 'germ_layer', 'subtype'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    obsm: 'X_harmony', 'X_pca', 'X_scvi', 'X_umap'

In [41]:
from sklearn.metrics import silhouette_score

silhouette_score(sample_data.X, sample_data.obs.seurat_clusters.tolist())

0.118041165

## 3. batch calculating

In [5]:
import pandas as pd

connection = pd.read_csv('/home/ouyangkang/data/trajectory_graph/celltype_connection.csv')

In [6]:
from sklearn.metrics import silhouette_score
import numpy as np

node_edge = []

for lineage in connection.iterrows():
    node_data = sc_data[np.in1d(sc_data.obs['subtype'], [lineage[1][0], lineage[1][1]])]

    #if len(set(node_data.obs.celltype.tolist())) != 2:
    #    print(f"here should include two cell type: {lineage[1][0]} and {lineage[1][1]}, but just one {set(node_data.obs.celltype.tolist())}")
    #else:
    edge_value = silhouette_score(node_data.X, node_data.obs.subtype.tolist())
    node_edge.append([lineage[1][0], lineage[1][1], edge_value])

In [7]:
pd.DataFrame(node_edge, columns=['Celltype_1', 'Celltype_2', 'Score']).to_csv('/home/ouyangkang/data/trajectory_graph/score.csv')

# 4. Scale the Silhouette score

In [22]:
import pandas as pd

data = pd.read_csv('/mnt/c/Users/ouyangkang/Desktop/score_1.csv')

In [23]:
data.columns = ['Source', 'Target', 'Weight']

In [24]:
data.head()

,Source,Target,Weight
0,stem cell,Malpighian tubule anlage,0.001000
1,Malpighian tubule anlage,Malpighian tubule primordium,0.118041
2,Malpighian tubule primordium,Malpighian tubule,0.033279
3,stem cell,hindgut anlage,0.001000
4,hindgut anlage,foregut/hindgut primordium,0.115821


In [25]:
data.Weight = list(map(lambda x:  0.0001 + (x - min(data.Weight)) / (max(data.Weight)-min(data.Weight)), data.Weight.tolist()))

In [26]:
data

,Source,Target,Weight
0,stem cell,Malpighian tubule anlage,0.146722
1,Malpighian tubule anlage,Malpighian tubule primordium,0.563000
2,Malpighian tubule primordium,Malpighian tubule,0.261530
3,stem cell,hindgut anlage,0.146722
4,hindgut anlage,foregut/hindgut primordium,0.555104
5,foregut/hindgut primordium,large intestine,0.826736
6,large intestine,hindgut,0.305464
7,stem cell,tracheal anlage,0.146722
8,tracheal anlage,tracheal primordium,0.500017
9,tracheal primordium,dorsal trunk,0.634655


In [27]:
data.to_csv('/mnt/c/Users/ouyangkang/Desktop/node_weight.csv', index=None)

In [13]:
graph_data = []
for index, target in data.iterrows():
    nodes = target[0] + ',' + target[1]
    graph_data.append([nodes, target[2]])
